In [ ]:
import Adafruit_DHT
import RPi.GPIO as GPIO
import time
import joblib
import numpy as np

# Load pre-trained Decision Tree model
model = joblib.load('irrigation_model.pkl')

# GPIO setup
DHT_SENSOR = Adafruit_DHT.DHT11
DHT_PIN = 4
PUMP_RELAY_PIN = 17

GPIO.setmode(GPIO.BCM)
GPIO.setup(PUMP_RELAY_PIN, GPIO.OUT)
GPIO.output(PUMP_RELAY_PIN, GPIO.HIGH)  # Ensure pump is off initially

def read_soil_moisture():
    # Replace this with actual ADC sensor reading if using MCP3008 or similar
    return 620  # Simulated soil moisture value

try:
    while True:
        humidity, temperature = Adafruit_DHT.read(DHT_SENSOR, DHT_PIN)
        soil_moisture = read_soil_moisture()

        print(f"Soil Moisture: {soil_moisture}")
        print(f"Temperature: {temperature}°C")
        print(f"Humidity: {humidity}%")

        # Prepare features for model prediction
        features = np.array([[temperature, humidity, soil_moisture]])
        prediction = model.predict(features)[0]

        if prediction == 1:
            GPIO.output(PUMP_RELAY_PIN, GPIO.LOW)  # Turn ON pump
            print("AI Prediction: Irrigation Needed\nPump Status: ON")
        else:
            GPIO.output(PUMP_RELAY_PIN, GPIO.HIGH)  # Turn OFF pump
            print("AI Prediction: No Irrigation Needed\nPump Status: OFF")

        time.sleep(5)

except KeyboardInterrupt:
    print("Exiting program...")

finally:
    GPIO.cleanup()
